# Library

In [1]:
from sklearn.tree import DecisionTreeClassifier  # sotring whole algo
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
#tree #KNN #Log #GAussinab #Rf #SVC #SVM
from sys import argv
from sklearn import metrics
import numpy as np   # array
import pandas as pd  # read + plot
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import classification_report,confusion_matrix #report
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import LabelEncoder #convert data
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
import pandas as pd
from imblearn.over_sampling import ADASYN, SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours
from imblearn.pipeline import Pipeline
from imblearn.datasets import make_imbalance
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tdf1= pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/ALL test F.csv')

#train
df1= pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/ALL F features in one.csv")

X_train1 = df1.drop('Target', axis=1)
y_train1 = df1['Target']
#--------------------------------------------

X_test1 = tdf1.drop('Target', axis=1)
y_test1 = tdf1['Target']
#--------------------------------------------


# **GRU model cv**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization, Activation
from keras.metrics import AUC
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score, roc_auc_score, confusion_matrix, average_precision_score

# Load the data
tdf1 = pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M test file.csv')
df1 = pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M-traine.csv")

# Combine train and test data for cross-validation
X = pd.concat([df1.drop('Target', axis=1), tdf1.drop('Target', axis=1)], axis=0)
y = pd.concat([df1['Target'], tdf1['Target']], axis=0)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create a GRU modelG with additional layers
modelG = Sequential()

modelG.add(Conv1D(filters=64, kernel_size=3, input_shape=(X.shape[1], 1)))
modelG.add(BatchNormalization())
modelG.add(Activation('sigmoid'))
modelG.add(MaxPooling1D(pool_size=2))

modelG.add(Conv1D(filters=128, kernel_size=3))
modelG.add(BatchNormalization())
modelG.add(Activation('relu'))
modelG.add(MaxPooling1D(pool_size=2))

modelG.add(Conv1D(filters=64, kernel_size=3))
modelG.add(BatchNormalization())
modelG.add(Activation('relu'))
modelG.add(MaxPooling1D(pool_size=2))

modelG.add(GRU(128, return_sequences=True))
modelG.add(GRU(64))
modelG.add(Dense(64))
modelG.add(BatchNormalization())
modelG.add(Activation('relu'))
modelG.add(Dropout(0.5))

modelG.add(Dense(1, activation='sigmoid'))

# Compile the modelG
modelG.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC()])

# Reshape data for GRU
X = X.reshape(X.shape[0], X.shape[1], 1)

# Initialize K-fold cross-validation
n_splits = 10  # You can adjust the number of splits
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store evaluation metrics
accuracies = []
mccs = []
kappas = []
specificities = []
sensitivities = []
aurocs = []
average_precisions = []

# Perform K-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the modelG
    modelG.fit(X_train, y_train, epochs=100, batch_size=128, verbose=0)

    # Generate predictions on the test data
    y_pred = modelG.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_binary)
    mcc = matthews_corrcoef(y_test, y_pred_binary)
    kappa = cohen_kappa_score(y_test, y_pred_binary)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    auroc = roc_auc_score(y_test, y_pred)
    average_precision = average_precision_score(y_test, y_pred)

    # Append metrics to lists
    accuracies.append(accuracy)
    mccs.append(mcc)
    kappas.append(kappa)
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    aurocs.append(auroc)
    average_precisions.append(average_precision)

# Print average metrics
print(f'Average Accuracy: {np.mean(accuracies):.4f}')
print(f'Average MCC: {np.mean(mccs):.4f}')
print(f'Average Kappa Score: {np.mean(kappas):.4f}')
print(f'Average Specificity: {np.mean(specificities):.4f}')
print(f'Average Sensitivity: {np.mean(sensitivities):.4f}')
print(f'Average AUROC: {np.mean(aurocs):.4f}')
print(f'Average Average Precision: {np.mean(average_precisions):.4f}')


10/10 [==============================] - 0s 39ms/step


# CNN-bIlstm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization, Activation, LSTM, Bidirectional, Dense, Dropout
from keras.metrics import AUC
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score, roc_auc_score, confusion_matrix, average_precision_score

# Load the data
# Load the data
tdf1 = pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M test file.csv')
df1 = pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M-traine.csv")

# Combine train and test data for cross-validation
X = pd.concat([df1.drop('Target', axis=1), tdf1.drop('Target', axis=1)], axis=0)
y = pd.concat([df1['Target'], tdf1['Target']], axis=0)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create a CNN-BiLSTM model
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, input_shape=(X.shape[1], 1)))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC()])

# Reshape data for CNN-BiLSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Initialize K-fold cross-validation
n_splits = 10  # You can adjust the number of splits
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store evaluation metrics
accuracies = []
mccs = []
kappas = []
specificities = []
sensitivities = []
aurocs = []
average_precisions = []

# Perform K-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=0)

    # Generate predictions on the test data
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_binary)
    mcc = matthews_corrcoef(y_test, y_pred_binary)
    kappa = cohen_kappa_score(y_test, y_pred_binary)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    auroc = roc_auc_score(y_test, y_pred)
    average_precision = average_precision_score(y_test, y_pred)

    # Append metrics to lists
    accuracies.append(accuracy)
    mccs.append(mcc)
    kappas.append(kappa)
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    aurocs.append(auroc)
    average_precisions.append(average_precision)

# Print average metrics
print(f'Average Accuracy: {np.mean(accuracies):.4f}')
print(f'Average MCC: {np.mean(mccs):.4f}')
print(f'Average Kappa Score: {np.mean(kappas):.4f}')
print(f'Average Specificity: {np.mean(specificities):.4f}')
print(f'Average Sensitivity: {np.mean(sensitivities):.4f}')
print(f'Average AUROC: {np.mean(aurocs):.4f}')
print(f'Average Average Precision: {np.mean(average_precisions):.4f}')


# GRU+**LSTM**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization, Activation, LSTM, GRU, Bidirectional, Dense, Dropout
from keras.metrics import AUC
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score, roc_auc_score, confusion_matrix, average_precision_score

# Load the data
tdf1 = pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M test file.csv')
df1 = pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M-traine.csv")

# Combine train and test data for cross-validation
X = pd.concat([df1.drop('Target', axis=1), tdf1.drop('Target', axis=1)], axis=0)
y = pd.concat([df1['Target'], tdf1['Target']], axis=0)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create a model with both GRU and LSTM layers
modelGL = Sequential()

modelGL.add(Conv1D(filters=64, kernel_size=3, input_shape=(X.shape[1], 1)))
modelGL.add(BatchNormalization())
modelGL.add(Activation('sigmoid'))
modelGL.add(MaxPooling1D(pool_size=2))
modelGL.add(Dropout(0.5))

modelGL.add(Conv1D(filters=64, kernel_size=3))
modelGL.add(BatchNormalization())
modelGL.add(Activation('relu'))
modelGL.add(MaxPooling1D(pool_size=2))

modelGL.add(GRU(256, return_sequences=True))
modelGL.add(LSTM(128, return_sequences=True))
modelGL.add(Dropout(0.5))
modelGL.add(GRU(64, return_sequences=True))
modelGL.add(LSTM(32, return_sequences=True))
modelGL.add(Flatten())
modelGL.add(Dense(64))
modelGL.add(BatchNormalization())
modelGL.add(Activation('relu'))
modelGL.add(Dropout(0.5))

modelGL.add(Dense(1, activation='sigmoid'))

# Compile the modelGL
modelGL.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC()])

# Reshape data for GRU+LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Initialize K-fold cross-validation
n_splits = 10  # You can adjust the number of splits
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store evaluation metrics
accuracies = []
mccs = []
kappas = []
specificities = []
sensitivities = []
aurocs = []
average_precisions = []

# Perform K-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the modelGL
    modelGL.fit(X_train, y_train, epochs=10, batch_size=128, verbose=0)

    # Generate predictions on the test data
    y_pred = modelGL.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_binary)
    mcc = matthews_corrcoef(y_test, y_pred_binary)
    kappa = cohen_kappa_score(y_test, y_pred_binary)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    auroc = roc_auc_score(y_test, y_pred)
    average_precision = average_precision_score(y_test, y_pred)

    # Append metrics to lists
    accuracies.append(accuracy)
    mccs.append(mcc)
    kappas.append(kappa)
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    aurocs.append(auroc)
    average_precisions.append(average_precision)

# Print average metrics
print(f'Average Accuracy: {np.mean(accuracies):.4f}')
print(f'Average MCC: {np.mean(mccs):.4f}')
print(f'Average Kappa Score: {np.mean(kappas):.4f}')
print(f'Average Specificity: {np.mean(specificities):.4f}')
print(f'Average Sensitivity: {np.mean(sensitivities):.4f}')
print(f'Average AUROC: {np.mean(aurocs):.4f}')
print(f'Average Average Precision: {np.mean(average_precisions):.4f}')


# RNN+CNN+GRU+LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization, Activation, SimpleRNN, LSTM, GRU, Dense, Dropout, Input
from keras.metrics import AUC
from sklearn.metrics import accuracy_score, matthews_corrcoef, cohen_kappa_score, roc_auc_score, confusion_matrix, average_precision_score

# Load the data
tdf1 = pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M test file.csv')
df1 = pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M-traine.csv")

# Combine train and test data for cross-validation
X = pd.concat([df1.drop('Target', axis=1), tdf1.drop('Target', axis=1)], axis=0)
y = pd.concat([df1['Target'], tdf1['Target']], axis=0)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create a model with SimpleRNN, LSTM, CNN, GRU, and additional layers
modelR = Sequential()

# Convolutional layers (CNN)
modelR.add(Conv1D(filters=64, kernel_size=3, input_shape=(X.shape[1], 1)))
modelR.add(BatchNormalization())
modelR.add(Activation('relu'))
modelR.add(MaxPooling1D(pool_size=2))
modelR.add(Dropout(0.5))

# RNN layers (SimpleRNN, LSTM, and GRU)
modelR.add(SimpleRNN(64, return_sequences=True))
modelR.add(BatchNormalization())
modelR.add(Dropout(0.5))
modelR.add(LSTM(64, return_sequences=True))
modelR.add(GRU(64))
modelR.add(Dropout(0.5))
modelR.add(Flatten())      # Flatten layer

# Fully connected layers
modelR.add(Dense(64))
modelR.add(Activation('relu'))
modelR.add(Dropout(0.5))

# Output layer
modelR.add(Dense(1, activation='sigmoid'))

# Compile the modelR
modelR.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC()])

# Reshape data for the modelR
X = X.reshape(X.shape[0], X.shape[1], 1)

# Initialize K-fold cross-validation
n_splits = 10  # You can adjust the number of splits
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store evaluation metrics
accuracies = []
mccs = []
kappas = []
specificities = []
sensitivities = []
aurocs = []
average_precisions = []

# Perform K-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the modelR
    modelR.fit(X_train, y_train, epochs=10, batch_size=128, verbose=0)

    # Generate predictions on the test data
    y_pred = modelR.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_binary)
    mcc = matthews_corrcoef(y_test, y_pred_binary)
    kappa = cohen_kappa_score(y_test, y_pred_binary)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    auroc = roc_auc_score(y_test, y_pred)
    average_precision = average_precision_score(y_test, y_pred)

# Append metrics to lists
    accuracies.append(accuracy)
    mccs.append(mcc)
    kappas.append(kappa)
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    aurocs.append(auroc)
    average_precisions.append(average_precision)

# Print average metrics
print(f'Average Accuracy: {np.mean(accuracies):.4f}')
print(f'Average MCC: {np.mean(mccs):.4f}')
print(f'Average Kappa Score: {np.mean(kappas):.4f}')
print(f'Average Specificity: {np.mean(specificities):.4f}')
print(f'Average Sensitivity: {np.mean(sensitivities):.4f}')
print(f'Average AUROC: {np.mean(aurocs):.4f}')
print(f'Average Average Precision: {np.mean(average_precisions):.4f}')


# ROC

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your data
tdf1 = pd.read_csv('/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M test file.csv')
df1 = pd.read_csv("/content/drive/MyDrive/a new RNA m6A/m5U/FULL/FUll separete dataset/M-traine.csv")
# Define more models and train them as needed

# Assuming you have already trained your models
models = {
    'CNN-BiLSTM': model,    # Define your 'model' here
    'CNN-RNN-GRU-LSTM': modelR,  # Define your 'modelR' here
    'GRU-LSTM': modelGL,  # Define your 'modelGL' here
    'GRUpred-m5U': modelG    # Define your 'modelG' here
}

# Split your data into features and labels
X = pd.concat([df1.drop('Target', axis=1), tdf1.drop('Target', axis=1)], axis=0)
y = pd.concat([df1['Target'], tdf1['Target']], axis=0)

# Encode your labels if they are not binary (0, 1)
# This step is optional if your labels are already binary
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=1042)

# Create a figure and axis for the ROC curve
plt.figure(figsize=(7, 4))

# For each model in your dictionary
for model_name, model in models.items():
    # Make predictions on the test set
    if hasattr(model, 'predict_proba'):
        y_pred = model.predict_proba(X_test)[:, 1]  # Use predict_proba for probability estimates
    else:
        y_pred = model.predict(X_test)
        y_pred = (y_pred - y_pred.min()) / (y_pred.max() - y_pred.min())  # Manually calculate probabilities

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    plt.plot(fpr, tpr)

# Plot the ROC curve for random guessing (AUC = 0.5)
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

# Set axis labels and title
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
# Save the resulting image as a JPEG file
plt.savefig('model1.jpg', format='jpg', dpi= 1500)
# Show the ROC curve plot
plt.show()